# Genshin YOLO detection Training Notebook

In this notebook, you will see how we train the Ultralytics YOLOv11 model to detect the most important elements visible in Genshin Impact. _(This notebook is designed for Google Colab but can be run anywhere with some adjustments)_

**First, let's verify that we have a GPU active**

In [ ]:
!nvidia-smi

We will use YOLOv11 (by Ultralytics), so we have to **install Ultralytics**

In [ ]:
!pip install -q ultralytics

# Get the data set from Github

In [ ]:
!apt-get install -y megatools
!megadl 'https://mega.nz/file/vRlEEQRA#lNBvZqg_MUG5MBn6XuvPb-za53pwwV6IuIu5guMicic'

In [ ]:
!unzip -q genshin-YOLO-database-uncompressed.zip -d /content/dataset

DATASET_PATH = '/content/dataset'
DATA_PATH = '/content/data.yaml'

# Create data.yaml

As we used LabelStudio to labelise our DataSet, we have a file ```classes.txt``` in our dataset and not a file ```data.yaml``` that is necessary to train our YOLO model. So, **let's create the file ```data.yaml```** using the file ```classes.txt```

In [ ]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names and index
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Write data to YAML file
  data = {
      'nc': number_of_classes,
      'names': classes
  }
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/dataset/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

# Split the data set

Now that we have our data and classes, we can split our data set into 3 subsets :
* Training set : Used to train the model.
* Validation set : Used to tune hyperparameters and prevent overfitting.
* Test set : Used to evaluate the final performance of the model.

For now, we are doing a random split to do that, but we are thinking of doing a stratified split based on this [article](https://jaidevd.com/posts/obj-detection-stratification/)

**First, we need to create a feature vector for each image to track the number of instances of each class present in the image**

## Feature Vectors

In [ ]:
import os
import shutil
import yaml
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

# ========================== Load Dataset and YAML ==================================
dataset_path = Path(DATASET_PATH)
labels = sorted(dataset_path.rglob("labels/*.txt"))  # All label files

yaml_file = DATA_PATH
with open(yaml_file, encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]  # YOLO class names

cls_idx = list(range(len(classes)))  # Class indices

# ============== Create Feature Vectors (Number of classes per image) ===============
index = [label.stem for label in labels]  # Use filenames (without extension) as IDs
labels_df = pd.DataFrame([], columns=cls_idx, index=index)

for label in labels:
    lbl_counter = Counter()

    with open(label) as lf:
        lines = lf.readlines()

    for line in lines:
        lbl_counter[int(line.split(" ")[0])] += 1  # Extract YOLO class index

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0).infer_objects(copy=False).astype(int)

import matplotlib.pyplot as plt

class_instances = labels_df.sum(axis=0)
class_names = [classes[idx] for idx in class_instances.index]

# Plot horizontal bar chart
plt.figure(figsize=(10, 6))
plt.barh(class_names, class_instances, color="skyblue")
plt.xlabel("Number of Instances")
plt.ylabel("Class Names")
plt.title("Instances per Class")
plt.gca().invert_yaxis()
plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.show()

sum(class_instances)

Now, we can split the dataset using these feature vectors
## Split

In [ ]:
validation_size = 0.15
test_size = 0.10
train_size = 1 - validation_size - test_size

# =============== Split Dataset into Train, Validation, and Test ====================
def split_dataset(labels_df, dataset_path, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, verbose=True):
    """
    Splits the dataset into train, validation, and test sets based on the provided ratios.
    Starting from the classes with less instances and in increasing order to ensure that the split respects the ratio.
    """
    assert train_ratio + val_ratio + test_ratio == 1, "Ratios must sum to 1"

    # Sort classes by number of instances (ascending order)
    sorted_index = class_instances.sort_values().index.tolist()
    train_ids, val_ids, test_ids = [], [], []

    for cls in sorted_index:
        cls_ids = labels_df[labels_df[cls] > 0].index.tolist()
        cls_train_ids, cls_temp_ids = train_test_split(cls_ids, train_size=train_ratio, random_state=42)
        cls_val_ids, cls_test_ids = train_test_split(cls_temp_ids, train_size=test_ratio / (val_ratio + test_ratio), random_state=42)

        train_ids.extend(cls_train_ids)
        val_ids.extend(cls_val_ids)
        test_ids.extend(cls_test_ids)

    # Remove duplicates between train, val, and test sets
    # with some order magic to keep ratios
    if train_ratio > val_ratio and train_ratio > test_ratio:
        train_ids = list(set(train_ids) - set(val_ids))
        val_ids = list(set(val_ids) - set(train_ids) - set(test_ids))
        test_ids = list(set(test_ids) - set(train_ids) - set(val_ids))
    else:
        val_ids = list(set(val_ids) - set(train_ids))
        train_ids = list(set(train_ids) - set(val_ids) - set(test_ids))
        test_ids = list(set(test_ids) - set(train_ids) - set(val_ids))

    if verbose:
        train_instances = labels_df.loc[train_ids,:].sum().sum()
        val_instances = labels_df.loc[val_ids,:].sum().sum()
        test_instances = labels_df.loc[test_ids,:].sum().sum()
        total_instances = train_instances + val_instances + test_instances
        total_train = len(train_ids)
        total_val = len(val_ids)
        total_test = len(test_ids)
        total = total_train + total_val + total_test

        print("\nWanted split:")
        print(f"Train samples: {total * train_ratio} ({train_ratio:.2%})")
        print(f"Validation samples: {total * val_ratio} ({val_ratio:.2%})")
        print(f"Test samples: {total * test_ratio} ({test_ratio:.2%})")

        # Distribution of instances
        print("\nInstance distribution per set:")
        print(f"Train instances: {train_instances} ({train_instances / total_instances:.2%})")
        print(f"Validation instances: {val_instances} ({val_instances / total_instances:.2%})")
        print(f"Test instances: {test_instances} ({test_instances / total_instances:.2%})")

        # Distribution of images
        print(f"\nTotal images: {total}")
        print("\nImage split:")
        print(f"Train samples: {total_train} ({total_train / total:.2%})")
        print(f"Validation samples: {total_val} ({total_val / total:.2%})")
        print(f"Test samples: {total_test} ({total_test / total:.2%})")

    return train_ids, val_ids, test_ids

train_files, val_files, test_files = split_dataset(labels_df, dataset_path,train_ratio=train_size,val_ratio=validation_size,test_ratio=test_size,verbose=True)


In [ ]:
# ======================= Create output folder =============================
output_path = Path("split_dataset")
for split in ["train", "val", "test"]:
    (output_path / split / "images").mkdir(parents=True, exist_ok=True)
    (output_path / split / "labels").mkdir(parents=True, exist_ok=True)

def move_files(files, split):
    for file in files:
        src_img = dataset_path / f"images/{file}.png"
        src_lbl = dataset_path / f"labels/{file}.txt"

        dst_img = output_path / split / "images" / f"{file}.png"
        dst_lbl = output_path / split / "labels" / f"{file}.txt"

        if src_img.exists():
            shutil.copy(src_img, dst_img)
        if src_lbl.exists():
            shutil.copy(src_lbl, dst_lbl)

move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

# ======================= Create data_split.yaml =============================
yaml_content = {
    "train": str("train"),
    "val": str("val"),
    "test": str("test"),
    "nc": len(classes),
    "names": classes,
}

with open(output_path / "data_split.yaml", "w") as f:
    yaml.dump(yaml_content, f)

print("Dataset successfully split into train, val, and test with stratified distribution.")

## Split distributions

Is our split good ? Let's see that

In [ ]:
# ===================== Plot Split Distributions =====================
import numpy as np

# Function to compute class densities (proportions)
def compute_class_densities(file_list):
    subset_df = labels_df.loc[file_list]  # Filter dataset for the split
    class_counts = subset_df.sum(axis=0)  # Get total instances per class
    total_instances = labels_df.sum(axis=0)  # Sum across all classes

    class_densities = np.zeros(len(classes))
    for i in range(len(classes)):
        if total_instances[i] != 0: # Avoid 0-Div
            class_densities[i] = class_counts[i] / total_instances[i]
        else:
            class_densities[i] = 0
    return class_densities

# Compute densities for each split
train_densities = compute_class_densities(train_files)
val_densities = compute_class_densities(val_files)
test_densities = compute_class_densities(test_files)

# Create a horizontal bar chart with normalized densities
y = np.arange(len(classes))  # Class indices
width = 0.25  # Bar width

plt.figure(figsize=(12, 8))

# Plot the bars horizontally
plt.barh(y - width, train_densities, height=width, label="Train", color="blue", alpha=0.7)
plt.barh(y, val_densities, height=width, label="Validation", color="orange", alpha=0.7)
plt.barh(y + width, test_densities, height=width, label="Test", color="green", alpha=0.7)

# Add labels and title
plt.ylabel("Class Names")
plt.xlabel("Proportion")
plt.title("Normalized Class Distribution in Train, Validation, and Test Sets")
plt.yticks(ticks=y, labels=classes)  # Set class names on the y-axis
plt.legend()
plt.grid(axis="x", linestyle="--", alpha=0.7)

# Show the plot
plt.show()

# Yolo call for training

In [ ]:
!yolo detect train data=/content/split_dataset/data_split.yaml model=yolo11n.pt epochs=20 imgsz=640

Testing with the validation folder

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=/content/split_dataset/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

We save the best weights

In [ ]:
from google.colab import files
!zip -r train.zip runs/detect/train/
files.download('train.zip')

We can now use the **best.pt** file to run the model in real time.